In [6]:
import pandas as pd
from docx import Document

def find_highlighted_cells(doc_path):
    doc = Document(doc_path)
    highlighted_cells = []

    # Iterate through tables
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        if run.font.highlight_color:  # Check if text is highlighted
                            highlighted_cells.append([cell.text])
                            break  # Stop checking further runs in the same cell

    return highlighted_cells

def save_to_csv(data, output_csv):
    df = pd.DataFrame(data, columns=["Highlighted Text"])
    df.to_csv(output_csv, index=False, encoding="utf-8")
    print(f"Highlighted text saved to {output_csv}")

doc_path = "LA.docx"
output_csv = "highlighted_cells.csv"

highlighted_cells = find_highlighted_cells(doc_path)
save_to_csv(highlighted_cells, output_csv)


Highlighted text saved to highlighted_cells.csv
